In [3]:
import numpy as np
import lmdb
import numpy as np
import torch
import six
from torch.utils.data import Dataset
import json
from PIL import Image

import matplotlib.pyplot as plt
from pathlib import Path
import cv2

import albumentations as A
from albumentations import KeypointParams
from albumentations.pytorch import ToTensorV2

In [9]:
Path().cwd().parents[2]

PosixPath('/Users/nakagawaayato/compe/kaggle/mga')

In [12]:
ROOT_DIR = Path().cwd().parents[2]
LMDB_DIR = ROOT_DIR / 'data' / 'data0004' / 'lmdb'

class cfg:
    input_size = 3
    output_size = 1
    img_h = 300
    img_w = 500
    heatmap_h = 80
    heatmap_w = 218
    sigma = 3

In [25]:
# Lmdb Dataset
class MgaLmdbDataset(Dataset):
    def __init__(self, cfg, lmdb_dir, indices, transforms):
        super().__init__()
        self.cfg = cfg
        self.transforms = transforms
        self.indices = indices
        self.env = lmdb.open(str(lmdb_dir), max_readers=32, readonly=True, lock=False, readahead=False, meminit=False)
        self.output_size = cfg.output_size
        self.sigma = cfg.sigma
        self.img_h, self.img_w = cfg.img_h, cfg.img_w
        self.heatmap_h, self.heatmap_w = cfg.heatmap_h, cfg.heatmap_w
        self.chart2point_name = {
            'scatter': 'scatter points',
            'line': 'lines',
            'dot': 'dot points',
            'vertical_bar': 'bars',
            'horizontal_bar': 'bars',
        }
    def _overlap_heatmap(self, heatmap, center, sigma):
        tmp_size = sigma * 6
        mu_x = int(center[0] + 0.5)
        mu_y = int(center[1] + 0.5)
        w, h = heatmap.shape[0], heatmap.shape[1]
        ul = [int(mu_x - tmp_size), int(mu_y - tmp_size)]
        br = [int(mu_x + tmp_size + 1), int(mu_y + tmp_size + 1)]
        if ul[0] >= h or ul[1] >= w or br[0] < 0 or br[1] < 0:
            return heatmap
        size = 2 * tmp_size + 1
        x = np.arange(0, size, 1, np.float32)
        y = x[:, np.newaxis]
        x0 = y0 = size // 2
        g = np.exp(- ((x - x0) ** 2 + (y - y0) ** 2) / (2 * sigma ** 2))
        g_x = max(0, -ul[0]), min(br[0], h) - ul[0]
        g_y = max(0, -ul[1]), min(br[1], w) - ul[1]
        img_x = max(0, ul[0]), min(br[0], h)
        img_y = max(0, ul[1]), min(br[1], w)
        heatmap[img_y[0]:img_y[1], img_x[0]:img_x[1]] = np.maximum(
        heatmap[img_y[0]:img_y[1], img_x[0]:img_x[1]],
        g[g_y[0]:g_y[1], g_x[0]:g_x[1]])
        return heatmap

    def _create_heatmap(self, joints):
        '''
            joints: [(x1, y1), (x2, y2), ...]
            heatmap: size: (hm_h, hm_w)
        '''
        heatmap = np.zeros((self.heatmap_h, self.heatmap_w), dtype=np.float32)
        for joint_id in range(len(joints)):
            heatmap = self._overlap_heatmap(heatmap, joints[joint_id], self.sigma)
        return heatmap
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        idx = self.indices[idx]
        with self.env.begin(write=False) as txn:
            # load image
            img_key = f'image-{str(idx+1).zfill(8)}'.encode()
            imgbuf = txn.get(img_key)

            # load json
            label_key = f'label-{str(idx+1).zfill(8)}'.encode()
            label = txn.get(label_key).decode('utf-8')
        
        # image
        buf = six.BytesIO()
        buf.write(imgbuf)
        buf.seek(0)
        if self.cfg.input_size == 3:
            img = np.array(Image.open(buf).convert('RGB'))
        else:
            img = np.array(Image.open(buf).convert('L'))
        
        # label
        json_dict = json.loads(label)
        chart_type = json_dict['chart-type']
        point_name = self.chart2point_name[chart_type]
        keypoints = [[dic['x'], dic['y']] for dic in json_dict['visual-elements'][point_name][0]]
        kp_arr = np.array(keypoints)
        kp_min = np.amin(kp_arr, 0)
        if kp_min[0] < 0 or kp_min[1] < 0:
            # print(keypoints)
            print(json_dict['id'])

        transformed = self.transforms(image=img, keypoints=keypoints)
        img = transformed['image']
        keypoints = transformed['keypoints']
        
        keypoints_on_hm = np.array(keypoints) * \
            np.array([self.heatmap_w, self.heatmap_h]) / np.array([self.img_w, self.img_h])

        heatmap = self._create_heatmap(keypoints_on_hm)

        img = torch.from_numpy(img).permute(2, 0, 1)
        heatmap = torch.from_numpy(heatmap)

        n_points = len(json_dict['data-series'])

        return img, heatmap, n_points

In [26]:
def get_transforms():
    return A.Compose([
        A.Resize(300, 500),
        A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ToTensorV2()
    ])

In [27]:
ds = MgaLmdbDataset(cfg, LMDB_DIR, [0, 1, 2, 3, 4], get_transforms())

In [28]:
ds.__getitem__(0)

ValueError: not enough values to unpack (expected 4, got 2)

In [15]:
env = lmdb.open(str(LMDB_DIR), max_readers=32, readonly=True, lock=False, readahead=False, meminit=False)
with env.begin(write=False) as txn:
    # load json
    label_key = f'label-{str(1).zfill(8)}'.encode()
    label = txn.get(label_key).decode('utf-8')
json_dict = json.loads(label)

In [24]:
json_dict['visual-elements']['scatter points'][0]

[{'x': 94.86666666666667, 'y': 209.98333333333335},
 {'x': 111.66666666666667, 'y': 194.78333333333336},
 {'x': 128.06666666666666, 'y': 187.98333333333335},
 {'x': 144.86666666666667, 'y': 182.38333333333335},
 {'x': 160.86666666666667, 'y': 203.18333333333334},
 {'x': 177.26666666666665, 'y': 179.58333333333334},
 {'x': 193.66666666666669, 'y': 130.78333333333336},
 {'x': 210.06666666666666, 'y': 142.38333333333335},
 {'x': 226.4666666666667, 'y': 199.98333333333335},
 {'x': 242.86666666666667, 'y': 141.18333333333334},
 {'x': 259.6666666666667, 'y': 166.38333333333335},
 {'x': 276.06666666666666, 'y': 78.78333333333336},
 {'x': 292.4666666666667, 'y': 139.58333333333334},
 {'x': 308.4666666666667, 'y': 196.38333333333335},
 {'x': 324.4666666666667, 'y': 139.58333333333334},
 {'x': 341.2666666666667, 'y': 200.78333333333336},
 {'x': 358.4666666666667, 'y': 159.58333333333334},
 {'x': 374.06666666666666, 'y': 188.38333333333335},
 {'x': 390.8666666666667, 'y': 202.38333333333335},
 {'